In [38]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np
import nltk
import string
import re
import time as t
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
import lyricsgenius
genius = lyricsgenius.Genius("bWUqXEIz9jor2w6fi9sLD3KfilReDFPZL7Ox3OCkD_Sz8ICXaAFSTOUkH-tsv1oW")
genius.remove_section_headers = True
genius.skip_non_songs = True

In [48]:
# The goal of this prototype is to fetch the lyrics of a specific song, in preparation for the full script that will iterate through the yearly data.

songdata = pd.read_csv('data/2019top10songs.csv')

In [49]:
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2228.0 Safari/537.3'}
cookies = { 'sessionid': '123..'}
i = 0 # Set i to the position in the songdata datafrae you would like to examine.
url = 'https://www.azlyrics.com/lyrics/'
artist = songdata['Artist'][i] # artist1 and artist2 are the most common ways azlyrics represents artist names. artist1 removes spaces while artist2 replaces                                  them with a dash.
artist1 = ''.join(artist.split()).lower().replace('\'','').replace('!','').replace('.','').replace('+','').replace('ñ','n').replace('é','e')
artist2 = artist.lower().replace('\'','').replace(' ','-').replace('!','').replace('.','').replace('+','').replace('ñ','n').replace('é','e')
artistl = []
if ', ' in artist: # Handles cases where there are three listed artists.
    artistl.append(artist1[:artist1.rfind(',')])
    artistl.append(artist1[artist1.find(',')+1:artist1.rfind('and')])
    artistl.append(artist1[artist1.rfind('and')+3:])
elif ' with ' in artist:
    artistl.append(artist1[:artist1.rfind('with')])
    artistl.append(artist2[:artist2.rfind('-with')])
elif ' and ' in artist: # Handles cases where there are two listed artists.
    artistl.append(artist1[:artist1.rfind('and')])
    artistl.append(artist2[:artist2.rfind('-and')])
    if ' featuring ' in artist: # Handles a case with multiple featured artists.
        artistl.append(artist1[:artist1.rfind('featuring')])
        artistl.append(artist2[:artist2.rfind('featuring')])
    artistl.append(artist1) # Cases where 'and' is in the artist name, ex. 'Tones and I'
elif 'bts' in artist1: # Handles that one k-pop group.
    artistl.append('bangtanboys')
elif ' featuring ' in artist: # Handles cases where there is a featured artist.
    artistl.append(artist1[:artist1.rfind('featuring')])
    artistl.append(artist2[:artist2.rfind('featuring')])
else: # Handles cases where there is one artist (which is most of them)
    artistl.append(artist1)
    artistl.append(artist2)
    artistl.append(artist1[artist1.find('the')+3:]) # Handles a rare case where Wikipedia includes a "the" in the artist name and azlyrics does not.
# The first three entries of the above list are standard single artist titles. The next three handle cases of collaborations between two artists. The next four after that handle collaborations between three artists. The last entry handles a rare case where Wikipedia includes a "the" in the artist name and azlyrics does not.
title = ''.join(songdata['Title'][i].split()).lower().replace('\'','').replace(',','').replace('!','').replace('ñ','n').replace('é','e') # azlyrics seems to represent all song titles in the same way.
titlel = [title, 'a' + title, 'the' + title] # However, there are very rare cases, such as 'Holly Jolly Christmas', where azlyrics includes an 'a' or a 'the' at the beginning of a song and Wikipedia does not. These cases will only be tested if all of the artist configurations fail.
if 'futsal' in title:
    titlel.insert(0,'futsalshuffle')
print(artistl)
print(titlel)

['mariahcarey', 'mariah-carey', 'riahcarey']
['alliwantforchristmasisyou', 'aalliwantforchristmasisyou', 'thealliwantforchristmasisyou']


In [41]:
geniusl = []
if ', ' in artist: # Handles cases where there are three listed artists.
    geniusl.append(artist[:artist.rfind(',')])
    geniusl.append(artist[artist.find(',')+1:artist1.rfind(' and ')])
    geniusl.append(artist[artist.rfind(' and ')+5:])
elif ' with ' in artist:
    geniusl.append(artist[:artist.rfind(' with')])
elif ' and ' in artist: # Handles cases where there are two listed artists.
    geniusl.append(artist[:artist.rfind(' and')])
    if ' featuring ' in artist: # Handles a case with multiple featured artists.
        geniusl.append(artist[:artist.rfind(' featuring ')])
    geniusl.append(artist) # Cases where 'and' is in the artist name, ex. 'Tones and I'
elif ' featuring ' in artist: # Handles cases where there is a featured artist.
    geniusl.append(artist[:artist.rfind(' featuring ')])
else: # Handles cases where there is one artist (which is most of them)
    if 'the' in artist:
        geniusl.append(artist[artist.find('the ')+4:])
    if 'The' in artist:
        geniusl.append(artist[artist.find('The ')+4:])
    geniusl.append(artist)
    if 'Show' in artist:
        geniusl.append(artist[artist.find('The ')+4:artist.rfind(' Show')]) # Handles some cases from the 50s and 60s.
geniusl

['Billy Preston']

In [50]:
success = False
for j in range(len(titlel)):
    for k in range(len(artistl)):
        if requests.get(url + artistl[k] + '/' + titlel[j] + '.html').status_code == 200:
            html = requests.get(url + artistl[k] + '/' + titlel[j] + '.html',cookies=cookies,headers=headers).text
            soup = BeautifulSoup(html, 'lxml')
            print(url + artistl[k] + '/' + titlel[j] + '.html')
            success = True
            t.sleep(3)
            break
        t.sleep(3)
    if success: # Escapes the outer loop when URL succeeds without using a second request.
        break
    if j == len(titlel) - 1:
        print('Song ' + songdata['Title'][i] + ' by ' + artist + ' at position ' + str(i) + ' not found!')

https://www.azlyrics.com/lyrics/mariahcarey/alliwantforchristmasisyou.html


In [42]:
success = False
for l in range(len(geniusl)):
    if genius.search_song(songdata['Title'][i].replace('\'',''),geniusl[l]) != None:
        currentsong = genius.search_song(songdata['Title'][i].replace('\'',''),geniusl[l])
        success = True
        t.sleep(3)
        break
    t.sleep(3)
if success == False:
    print('Song ' + songdata['Title'][i] + ' by ' + artist + ' at position ' + str(i) + ' not found!')

Searching for "Space Race" by Billy Preston...
Done.
Searching for "Space Race" by Billy Preston...
Done.


In [43]:
artistl

['billypreston', 'billy-preston', 'llypreston']

In [51]:
print(soup.prettify()[:10000])

<!DOCTYPE html>
<html lang="en">
 <head>
  <meta charset="utf-8"/>
  <meta content="IE=edge" http-equiv="X-UA-Compatible"/>
  <meta content="width=device-width, initial-scale=1" name="viewport"/>
  <meta content="Mariah Carey &quot;All I Want For Christmas Is You&quot;: I don't want a lot for Christmas There is just one thing I need I don't care about the presents Unde..." name="description"/>
  <meta content="All I Want For Christmas Is You lyrics, Mariah Carey All I Want For Christmas Is You lyrics, Mariah Carey lyrics" name="keywords"/>
  <meta content="noarchive" name="robots"/>
  <meta content="//www.azlyrics.com/az_logo_tr.png" property="og:image"/>
  <title>
   Mariah Carey - All I Want For Christmas Is You Lyrics | AZLyrics.com
  </title>
  <link href="https://maxcdn.bootstrapcdn.com/bootstrap/3.3.4/css/bootstrap.min.css" rel="stylesheet"/>
  <link href="//www.azlyrics.com/bsaz.css?4" rel="stylesheet"/>
  <!-- HTML5 shim and Respond.js for IE8 support of HTML5 elements and medi

In [52]:
my_text = soup.find('div',attrs={'class': None}).text.replace('\n',' ')[3:] # Extract lyrics text from the azlyrics page and replace newlines with spaces. Also, eliminate the \r tag and space at the beginning.
my_text = word_tokenize(my_text)
my_text = [i.lower() for i in my_text if not re.fullmatch('[' + string.punctuation + ']+', i)] #Remove tokens that are just punctuation.

In [45]:
my_text = currentsong.lyrics
my_text = word_tokenize(my_text)
my_text = [i.lower() for i in my_text if not re.fullmatch('[' + string.punctuation + ']+', i)] #Remove tokens that are just punctuation.
my_text

['all',
 'these',
 'songs',
 'are',
 'about',
 'or',
 'heavily',
 'reference',
 'historical',
 'events',
 'and',
 'are',
 'listed',
 'in',
 'alphabetical',
 'order',
 'by',
 'song',
 'title',
 'ca',
 "n't",
 'find',
 'your',
 'song',
 'add',
 'to',
 'this',
 'list',
 'and',
 'link',
 'it',
 'a',
 'aberfan',
 'by',
 'dulahan',
 'about',
 'the',
 '1966',
 'coal',
 'mine',
 'disaster',
 'in',
 'south',
 'wales',
 'the',
 'adams',
 'administration',
 'by',
 'lin-manuel',
 'miranda',
 'about',
 'alexander',
 'hamilton',
 'during',
 'john',
 'adams',
 'presidency',
 'agent',
 'orange',
 'by',
 'kamalata',
 'connects',
 'the',
 'use',
 'of',
 'agent',
 'orange',
 'to',
 'earlier',
 'u.s.',
 'war',
 'activities',
 'a',
 'great',
 'day',
 'for',
 'freedom',
 'by',
 'pink',
 'floyd',
 'about',
 'the',
 'aftermath',
 'of',
 'the',
 'berlin',
 'wall',
 'collapse',
 'a',
 'league',
 'of',
 'notions',
 'by',
 'al',
 'stewart',
 'about',
 'the',
 'league',
 'of',
 'nations',
 'alexander',
 'hamilton'

In [53]:
port_stem = PorterStemmer() # The following code will stem the words in the lyrics. This means that, for example, "stop", "stopping", and "stops" will only count as one unique word.

stem_text = []

for i in range(len(my_text)):
    stem_text.append(port_stem.stem(my_text[i]))

print(stem_text)

['i', 'do', "n't", 'want', 'a', 'lot', 'for', 'christma', 'there', 'is', 'just', 'one', 'thing', 'i', 'need', 'i', 'do', "n't", 'care', 'about', 'the', 'present', 'underneath', 'the', 'christma', 'tree', 'i', 'just', 'want', 'you', 'for', 'my', 'own', 'more', 'than', 'you', 'could', 'ever', 'know', 'make', 'my', 'wish', 'come', 'true', 'all', 'i', 'want', 'for', 'christma', 'is', 'you', 'yeah', 'i', 'do', "n't", 'want', 'a', 'lot', 'for', 'christma', 'there', 'is', 'just', 'one', 'thing', 'i', 'need', 'and', 'i', 'do', "n't", 'care', 'about', 'the', 'present', 'underneath', 'the', 'christma', 'tree', 'i', 'do', "n't", 'need', 'to', 'hang', 'my', 'stock', 'there', 'upon', 'the', 'fireplac', 'santa', 'clau', 'wo', "n't", 'make', 'me', 'happi', 'with', 'a', 'toy', 'on', 'christma', 'day', 'i', 'just', 'want', 'you', 'for', 'my', 'own', 'more', 'than', 'you', 'could', 'ever', 'know', 'make', 'my', 'wish', 'come', 'true', 'all', 'i', 'want', 'for', 'christma', 'is', 'you', 'you', 'babi', 'o

In [54]:
unique_count = len(set(stem_text))
unique_count

114

In [55]:
len(my_text)

334